In [2]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
query2=f"""
SELECT
*
FROM `perceptive-ivy-290216.f1_api.sprint_lap_time`  A
# WHERE A.Year=2024
# AND A.GP="Monaco Grand Prix"
# AND A.DRIVER='HAM'
ORDER BY LapNumber, LapStartTime
"""
track3=pd.read_gbq(query2,project_id,dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1906/99385975.py:10: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  track3=pd.read_gbq(query2,project_id,dialect='standard')


In [39]:
track2=track3[(track3["GP"]=='Austrian Grand Prix')&(track3["Year"]==2024)]
track2.head()
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]

In [40]:
track2['LapTime']= pd.to_timedelta(track2["LapTime"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1906/4173085553.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
# track2['LapTime'].dt.total_seconds().min()*1.07
track2=track2[track2['LapNumber']!=1.0]

In [42]:
quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
quicklaps.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
435,0 days 00:50:51.099000,VER,1,0 days 00:01:09.122000,2.0,1.0,NaT,NaT,0 days 00:00:17.572000,0 days 00:00:30.882000,0 days 00:00:20.668000,0 days 00:49:59.549000,0 days 00:50:30.431000,0 days 00:50:51.099000,292.0,237.0,274.0,295.0,True,MEDIUM,7.0,False,Red Bull Racing,0 days 00:49:41.977000,2024-06-29 10:07:16.703,1,1.0,False,,False,True,2024,Austrian Grand Prix
436,0 days 00:50:51.824000,NOR,4,0 days 00:01:08.935000,2.0,1.0,NaT,NaT,0 days 00:00:17.171000,0 days 00:00:31.117000,0 days 00:00:20.647000,0 days 00:50:00.102000,0 days 00:50:31.219000,0 days 00:50:51.866000,313.0,239.0,283.0,309.0,True,MEDIUM,6.0,False,McLaren,0 days 00:49:42.889000,2024-06-29 10:07:17.615,1,2.0,False,,False,True,2024,Austrian Grand Prix
437,0 days 00:50:52.662000,PIA,81,0 days 00:01:08.980000,2.0,1.0,NaT,NaT,0 days 00:00:17.326000,0 days 00:00:30.794000,0 days 00:00:20.860000,0 days 00:50:01.019000,0 days 00:50:31.813000,0 days 00:50:52.673000,317.0,238.0,281.0,315.0,True,MEDIUM,5.0,False,McLaren,0 days 00:49:43.682000,2024-06-29 10:07:18.408,1,3.0,False,,False,True,2024,Austrian Grand Prix
438,0 days 00:50:53.434000,SAI,55,0 days 00:01:09.263000,2.0,1.0,NaT,NaT,0 days 00:00:17.244000,0 days 00:00:31.086000,0 days 00:00:20.933000,0 days 00:50:01.474000,0 days 00:50:32.560000,0 days 00:50:53.493000,319.0,233.0,284.0,317.0,True,MEDIUM,5.0,False,Ferrari,0 days 00:49:44.171000,2024-06-29 10:07:18.897,1,4.0,False,,False,True,2024,Austrian Grand Prix
439,0 days 00:50:54.018000,RUS,63,0 days 00:01:09.349000,2.0,1.0,NaT,NaT,0 days 00:00:17.200000,0 days 00:00:31.264000,0 days 00:00:20.885000,0 days 00:50:01.892000,0 days 00:50:33.156000,0 days 00:50:54.041000,320.0,233.0,286.0,309.0,True,MEDIUM,5.0,False,Mercedes,0 days 00:49:44.669000,2024-06-29 10:07:19.395,1,5.0,False,,False,True,2024,Austrian Grand Prix


In [43]:
transformed_laps = quicklaps.copy()
transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

# order the team from the fastest (lowest median lap time) tp slower
team_order = (
    transformed_laps[["Driver", "LapTime (s)"]].groupby("Driver").median()["LapTime (s)"].sort_values().index
)
print(team_order)

Index(['VER', 'NOR', 'RUS', 'PIA', 'HAM', 'SAI', 'LEC', 'PER', 'MAG', 'STR',
       'OCO', 'GAS', 'RIC', 'TSU', 'HUL', 'ALO', 'SAR', 'ALB', 'BOT', 'ZHO'],
      dtype='object', name='Driver')


In [44]:
transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('median')
transformed_laps.tail()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime (s),median
5684,0 days 01:15:54.421000,ALO,14,0 days 00:01:13.198000,23.0,1.0,NaT,NaT,0 days 00:00:17.908000,0 days 00:00:33.227000,0 days 00:00:22.063000,0 days 01:14:59.147000,0 days 01:15:32.374000,0 days 01:15:54.437000,295.0,227.0,273.0,296.0,False,MEDIUM,29.0,False,Aston Martin,0 days 01:14:41.223000,2024-06-29 10:32:15.949,1,16.0,False,,False,True,2024,Austrian Grand Prix,73.198,70.8940
5685,0 days 01:15:55.342000,SAR,2,0 days 00:01:11.601000,23.0,1.0,NaT,NaT,0 days 00:00:17.634000,0 days 00:00:32.282000,0 days 00:00:21.685000,0 days 01:15:01.378000,0 days 01:15:33.660000,0 days 01:15:55.345000,307.0,231.0,275.0,305.0,False,MEDIUM,26.0,False,Williams,0 days 01:14:43.741000,2024-06-29 10:32:18.467,1,17.0,False,,False,True,2024,Austrian Grand Prix,71.601,70.9815
5686,0 days 01:15:55.939000,ALB,23,0 days 00:01:11.673000,23.0,1.0,NaT,NaT,0 days 00:00:17.488000,0 days 00:00:32.541000,0 days 00:00:21.644000,0 days 01:15:01.789000,0 days 01:15:34.330000,0 days 01:15:55.974000,324.0,223.0,283.0,320.0,False,MEDIUM,29.0,False,Williams,0 days 01:14:44.266000,2024-06-29 10:32:18.992,1,18.0,False,,False,True,2024,Austrian Grand Prix,71.673,71.1035
5687,0 days 01:15:57.777000,BOT,77,0 days 00:01:12.068000,23.0,1.0,NaT,NaT,0 days 00:00:17.742000,0 days 00:00:32.664000,0 days 00:00:21.662000,0 days 01:15:03.460000,0 days 01:15:36.124000,0 days 01:15:57.786000,295.0,220.0,274.0,297.0,False,MEDIUM,23.0,True,Kick Sauber,0 days 01:14:45.709000,2024-06-29 10:32:20.435,1,19.0,False,,False,True,2024,Austrian Grand Prix,72.068,71.2805
5688,0 days 01:16:04.409000,ZHO,24,0 days 00:01:12.108000,23.0,1.0,NaT,NaT,0 days 00:00:17.831000,0 days 00:00:32.572000,0 days 00:00:21.705000,0 days 01:15:10.140000,0 days 01:15:42.712000,0 days 01:16:04.417000,NaN,221.0,271.0,297.0,False,MEDIUM,23.0,True,Kick Sauber,0 days 01:14:52.301000,2024-06-29 10:32:27.027,1,20.0,False,,False,True,2024,Austrian Grand Prix,72.108,71.6530


In [45]:
fig=px.box(
    transformed_laps.sort_values(by=["median","LapNumber"]),
    x="Driver",
    y="LapTime (s)",
    color='Driver',
    template="xgridoff",
    title="<b>Driver Pace Plot for the {} {} Sprint Race</b>".format(year,gp),
    # palette=team_palette,
    # whiskerprops=dict(color="white"),
    # boxprops=dict(edgecolor="white"),
    # medianprops=dict(color="grey"),
    # capprops=dict(color="white"),
    height=700, width=1200,

color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)
fig.update_traces(opacity=1)
fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    margin=dict(l=50, r=5, t=35, b=50)
)
fig.show()

In [46]:
fig.write_html("{} {} Sprint Race Driver Pace Plot.html".format(year,gp))

In [47]:
#Get MINIMUM OF THE MEDIAN LAP TIMES

fastest_lap = transformed_laps["median"].min()
transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
transformed_laps_relative_median=transformed_laps[["Driver","Delta","Delta_Percent"]].drop_duplicates()
transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
transformed_laps_relative_median

,Driver,Delta,Delta_Percent
435,VER,0.0000,100.0%
436,NOR,0.0760,100.109%
439,RUS,0.1260,100.181%
437,PIA,0.1835,100.264%
440,HAM,0.2470,100.355%
438,SAI,0.2665,100.383%
441,LEC,0.4410,100.634%
442,PER,0.5750,100.826%
443,MAG,0.7815,101.123%
445,STR,0.9150,101.315%


In [48]:
for index, row in transformed_laps_relative_median.iterrows():
  if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
    transformed_laps_relative_median.loc[index, 'Delta']=0.001
transformed_laps_relative_median

,Driver,Delta,Delta_Percent
435,VER,0.0010,100.0%
436,NOR,0.0760,100.109%
439,RUS,0.1260,100.181%
437,PIA,0.1835,100.264%
440,HAM,0.2470,100.355%
438,SAI,0.2665,100.383%
441,LEC,0.4410,100.634%
442,PER,0.5750,100.826%
443,MAG,0.7815,101.123%
445,STR,0.9150,101.315%


In [49]:
fig=px.bar(
    transformed_laps_relative_median,
    x="Driver",
    y="Delta",
    color='Driver',
    template="xgridoff",
    title="<b>Driver Pace (Median Lap Times) for the {} {} Sprint Race</b>".format(year,gp),
    text="Delta_Percent",
    height=700, 
    width=1200,
    color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)

fig.update_traces(textposition='outside')

fig.update_traces(opacity=0.85)

fig.update_traces( marker_line_color='white',marker_line_width=1.5)

fig.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig.update_yaxes(range=[2.5, 0])

fig.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig.update_xaxes(title_text='')      # Remove axis titles
fig.update_yaxes(title_text='')

fig.update_xaxes(
        title_standoff = 75
)

fig.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
fig.show()

In [50]:
fig.write_html("{} {} Sprint Race Driver Median Plot.html".format(year,gp))

In [51]:
#Get MINIMUM OF ALL LAP TIMES
transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('min')

fastest_lap_all = transformed_laps["min_all"].min()
transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
transformed_laps_relative_min_all=transformed_laps[["Driver","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
transformed_laps_relative_min_all

,Driver,Delta_Min_All,Delta_Percent_All
436,NOR,0.000,100.0%
437,PIA,0.045,100.065%
435,VER,0.078,100.113%
438,SAI,0.186,100.27%
439,RUS,0.259,100.376%
440,HAM,0.330,100.479%
441,LEC,0.417,100.605%
442,PER,0.485,100.704%
443,MAG,1.007,101.461%
446,GAS,1.233,101.789%


In [52]:
for index, row in transformed_laps_relative_min_all.iterrows():
  if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
    transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
transformed_laps_relative_min_all

,Driver,Delta_Min_All,Delta_Percent_All
436,NOR,0.001,100.0%
437,PIA,0.045,100.065%
435,VER,0.078,100.113%
438,SAI,0.186,100.27%
439,RUS,0.259,100.376%
440,HAM,0.330,100.479%
441,LEC,0.417,100.605%
442,PER,0.485,100.704%
443,MAG,1.007,101.461%
446,GAS,1.233,101.789%


In [53]:
fig=px.bar(
    transformed_laps_relative_min_all,
    x="Driver",
    y="Delta_Min_All",
    color='Driver',
    template="xgridoff",
    title="<b>Driver Pace (Fastest Lap Times) for the {} {} Sprint Race</b>".format(year,gp),
    text="Delta_Percent_All",
    height=700, width=1200,
    color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)

fig.update_traces(textposition='outside')

fig.update_traces(opacity=0.85)

fig.update_traces( marker_line_color='white',marker_line_width=1.5)

fig.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig.update_yaxes(range=[4.2, 0])

fig.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig.update_xaxes(title_text='')      # Remove axis titles
fig.update_yaxes(title_text='')

fig.update_xaxes(
        title_standoff = 75
)

fig.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig.update_yaxes(ticksuffix = "  ")

fig.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

fig.show()

In [54]:
fig.write_html("{} {} Sprint Race Driver Fastest Lap Plot.html".format(year,gp))

In [55]:
# #Get AVERAGE OF ALL LAP TIMES
# transformed_laps['avg_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform(np.mean)

# fastest_lap_avg = transformed_laps["avg_all"].min()
# transformed_laps['Delta_Avg_All']= transformed_laps["avg_all"] - (fastest_lap_avg)
# transformed_laps['Delta_Percent_Avg']= (transformed_laps["avg_all"]/(fastest_lap_avg)-1)*100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].round(3)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].astype(str)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+"%"
# transformed_laps_relative_avg_all=transformed_laps[["Driver","Delta_Avg_All","Delta_Percent_Avg"]].drop_duplicates()
# transformed_laps_relative_avg_all=transformed_laps_relative_avg_all.sort_values("Delta_Avg_All")
# transformed_laps_relative_avg_all

In [56]:
# for index, row in transformed_laps_relative_avg_all.iterrows():
#   if transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']==0.0:
#     transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']=0.001
# transformed_laps_relative_avg_all

In [57]:
# fig=px.bar(
#     transformed_laps_relative_avg_all,
#     x="Driver",
#     y="Delta_Avg_All",
#     color='Driver',
#     template="xgridoff",
#     title="<b>Driver Pace (Average Lap Times) for the {} {}</b>".format(year,gp),
#     text="Delta_Percent_Avg",
#     height=700, width=1200,
#     color_discrete_map={
#                  "ALB": "#64C4FF",
#                  "ALO": "#229971",
#                  "BOT": "#52e252",
#                  "GAS": "#0093cc",
#                  "HAM": "#27F4D2",
#                  "HUL": "#B6BABD",
#                  "LEC": "#E80020",
#                  "MAG": "#B6BABD",
#                  "NOR": "#FF8000",
#                  "OCO": "#0093cc",
#                  "PER": "#3671C6",
#                  "PIA": "#FF8000",
#                  "RIC": "#6692FF",
#                  "RUS": "#27F4D2",
#                  "SAI": "#E80020",
#                  "SAR": "#37BEDD",
#                  "STR": "#229971",
#                  "TSU": "#6692FF",
#                  "VER": "#3671C6",
#                  "ZHO": "#52e252"
#           }
# )

# fig.update_traces(textposition='outside')

# fig.update_traces(opacity=0.85)

# fig.update_traces( marker_line_color='white',marker_line_width=1.5)

# fig.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_yaxes(range=[3.5, 0])

# fig.update_layout(
#     xaxis={'side': 'top'},
#     yaxis={'side': 'left'}
# )

# fig.update_xaxes(title_text='')      # Remove axis titles
# fig.update_yaxes(title_text='')

# fig.update_xaxes(
#         title_standoff = 75
# )

# fig.update_layout(
#     hoverlabel=dict(
#     bgcolor="white",
#     font_size=16,
#     font_family="PT Sans Narrow"
#     ),
#     title_x=0.5,
#     margin=dict(l=60, r=5, t=110, b=30),
#     yaxis = dict(tickfont = dict(size=15)),
#     xaxis = dict(tickfont = dict(size=15)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=12,
#         color="Black"
#     ),
#     title_font_family="PT Sans Narrow"
# )
# fig.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
# fig.show()

In [166]:
# fig.write_html("{} {} Driver Avg Lap Plot.html".format(year,gp))